In [1]:
'''
Author: ChenHJ
Date: 2022-12-08 12:39:00
LastEditors: ChenHJ
LastEditTime: 2022-12-08 13:02:22
FilePath: /chenhj/0302code/cal_tmpvar_wap_sst_season_dask.ipynb
Aim: 
12/08/2022
  1)本文件用于dask试验
Mission: 
'''
from mailbox import _PartialFile
import numpy as np
import xarray as xr
import os
import re
from cdo import *
import shutil
import sys

sys.path.append("/home/ys17-23/chenhj/self_def/")
import plot as sepl
import cal as ca
import pandas as pd
from importlib import reload

import metpy.calc as mpcalc
import metpy.constants as constants
import geocat.comp
from windspharm.xarray import VectorWind
import skill_metrics as sm
from brokenaxes import brokenaxes

reload(sepl)

# sd.path.append("/home/ys17-23/chenhj/1201code/self_def.py")

cdo = Cdo

# for plot
import proplot as pplt
import cartopy.crs as ccrs
import cartopy.feature as cfeature
from cartopy.mpl.ticker import LongitudeFormatter
from cartopy.mpl.ticker import LatitudeFormatter
from cartopy.util import add_cyclic_point
from matplotlib.ticker import MultipleLocator
from matplotlib.patches import Rectangle
import matplotlib.pyplot as plt
from scipy import stats
from scipy.stats import t
from scipy import signal
from eofs.multivariate.standard import MultivariateEof
from eofs.standard import Eof
import statsmodels.api as sm
import dask
from dask.distributed import Client
from dask.diagnostics import ProgressBar

In [ ]:
if __name__ == "__main__":
  with ProgressBar():
    client = Client(n_workers=1, threads_per_worker=1,memory_limit='100GB',processes=False)
    import dask.array as da
    x = da.random.random((10000, 10000, 10), chunks=(1000,1000,5))
    y = da.random.random((10000, 10000, 10), chunks=(1000,1000,5))
    z = (da.arcsin(x) + da.arccos(y)).sum(axis=(1,2))
    z.compute()
client.close()

In [12]:
toshis_ds = xr.open_dataarray("/home/ys17-23/Extension/personal-data/chenhj/SAM_EAM_data/CMIP6/historical/tmp_var/original/non_detrend/tos_historical_r144x72_195001-201412.nc", chunks=dict(time=-1,lat=36))
tosssp585_ds = xr.open_dataarray("/home/ys17-23/Extension/personal-data/chenhj/SAM_EAM_data/CMIP6/ssp585/tmp_var/original/non_detrend/tos_ssp585_r144x72_201501-209912.nc", chunks=dict(time=-1,lat=36))


In [43]:
if __name__ == "__main__":
  with dask.config.set(**{'array.slicing.split_large_chunks': True}):
    client = Client(n_workers=6, threads_per_worker=5,memory_limit='50GB',processes=False)
    print(client.dashboard_link)
    with ProgressBar():
      tosCMIP6_ds = xr.concat([toshis_ds.sel(time=toshis_ds.time.dt.year>=1979), tosssp585_ds], "time")
      tosCMIP6_trend, tosCMIP6_trend_pvalue, tosCMIP6_ds_detrend = ca.new_detrend_dim(tosCMIP6_ds.chunk(dict(time=-1,models=13)))
      tosCMIP6_ds_detrend.name = "sst"
      tosCMIP6_ds_detrend_tmp = tosCMIP6_ds_detrend.compute()
client.close()

/home/soft/anaconda3/envs/chenhj_test_cartopy/lib/python3.8/site-packages/distributed/node.py:182: UserWarning: Port 8787 is already in use.
Perhaps you already have a cluster running?
Hosting the HTTP server on port 42563 instead
  warnings.warn(


http://172.16.102.35:42563/status


In [44]:
tosCMIP6_ds_detrend_tmp

<xarray.DataArray 'sst' (models: 26, time: 1452, lat: 72, lon: 144)>
array([[[[            nan,             nan,             nan, ...,
                      nan,             nan,             nan],
         [            nan,             nan,             nan, ...,
                      nan,             nan,             nan],
         [            nan,             nan,             nan, ...,
                      nan,             nan,             nan],
         ...,
         [ 5.05188548e-01,  5.28957426e-01,  5.56833324e-01, ...,
           4.30803660e-01,  4.58953959e-01,  4.83238590e-01],
         [ 5.01579520e-01,  5.04743176e-01,  5.07132014e-01, ...,
           4.88646952e-01,  4.93279178e-01,  4.97731897e-01],
         [ 4.80539447e-01,  4.81348283e-01,  4.82151379e-01, ...,
           4.77787103e-01,  4.78880497e-01,  4.79724404e-01]],

        [[            nan,             nan,             nan, ...,
                      nan,             nan,             nan],
         [            nan,             nan,             nan, ...,
                      nan,             nan,             nan],
         [            nan,             nan,             nan, ...,
                      nan,             nan,             nan],
...
         [ 2.08506060e+00,  2.11150364e+00,  2.12765148e+00, ...,
           2.10098919e+00,  2.05982048e+00,  2.05122697e+00],
         [ 2.26362646e+00,  2.24440443e+00,  2.21963277e+00, ...,
           2.25277558e+00,  2.25778305e+00,  2.26212525e+00],
         [ 2.47057660e+00,  2.46525810e+00,  2.46013189e+00, ...,
           2.48610702e+00,  2.48141722e+00,  2.47609474e+00]],

        [[            nan,             nan,             nan, ...,
                      nan,             nan,             nan],
         [            nan,             nan,             nan, ...,
                      nan,             nan,             nan],
         [            nan,             nan,             nan, ...,
                      nan,             nan,             nan],
         ...,
         [ 5.04814499e-01,  5.16649665e-01,  5.18064660e-01, ...,
           4.95378345e-01,  4.88564353e-01,  4.87668698e-01],
         [ 7.04925025e-01,  6.94418351e-01,  6.81908276e-01, ...,
           7.35243274e-01,  7.23569407e-01,  7.14402693e-01],
         [ 1.07687881e+00,  1.07142577e+00,  1.06615695e+00, ...,
           1.08929939e+00,  1.08706024e+00,  1.08252225e+00]]]])
Coordinates:
  * time     (time) datetime64[ns] 1979-01-01 1979-02-01 ... 2099-12-01
  * lon      (lon) float64 0.0 2.5 5.0 7.5 10.0 ... 350.0 352.5 355.0 357.5
  * lat      (lat) float64 -88.75 -86.25 -83.75 -81.25 ... 83.75 86.25 88.75
  * models   (models) object 'ACCESS-CM2' 'BCC-CSM2-MR' ... 'UKESM1-0-LL'
Attributes:
    standard_name:  sea_surface_temperature
    long_name:      Sea Surface Temperature
    units:          degC
    comment:        Temperature of upper boundary of the liquid ocean, includ...
    cell_methods:   area: mean where sea time: mean
    cell_measures:  area: areacello
    history:        2019-11-08T18:45:39Z altered by CMOR: replaced missing va...

In [50]:
tosCMIP6_ds_detrend_tmp.sel(time=tosCMIP6_ds_detrend.time.dt.year<=2014).chunk(dict(time=-1,lon=72,lat=36))

<xarray.DataArray 'sst' (models: 26, time: 432, lat: 72, lon: 144)>
dask.array<xarray-<this-array>, shape=(26, 432, 72, 144), dtype=float64, chunksize=(26, 432, 36, 72), chunktype=numpy.ndarray>
Coordinates:
  * time     (time) datetime64[ns] 1979-01-01 1979-02-01 ... 2014-12-01
  * lon      (lon) float64 0.0 2.5 5.0 7.5 10.0 ... 350.0 352.5 355.0 357.5
  * lat      (lat) float64 -88.75 -86.25 -83.75 -81.25 ... 83.75 86.25 88.75
  * models   (models) object 'ACCESS-CM2' 'BCC-CSM2-MR' ... 'UKESM1-0-LL'
Attributes:
    standard_name:  sea_surface_temperature
    long_name:      Sea Surface Temperature
    units:          degC
    comment:        Temperature of upper boundary of the liquid ocean, includ...
    cell_methods:   area: mean where sea time: mean
    cell_measures:  area: areacello
    history:        2019-11-08T18:45:39Z altered by CMOR: replaced missing va...

In [51]:
if __name__ == "__main__":
  with dask.config.set(**{'array.slicing.split_large_chunks': True}):
    client = Client(n_workers=4, threads_per_worker=5,memory_limit='50GB',processes=False)
    print(client.dashboard_link)
    with ProgressBar():
      toshis_ds_trend, toshis_ds_trend_pvalue, toshis_ds_detrend = ca.new_detrend_dim(tosCMIP6_ds_detrend_tmp.sel(time=tosCMIP6_ds_detrend_tmp.time.dt.year<=2014).chunk(time=-1,lon=72,lat=36))
      toshis_ds_detrend.name = "sst"
      toshis_ds_detrend_copy = toshis_ds_detrend.compute()
client.close()

/home/soft/anaconda3/envs/chenhj_test_cartopy/lib/python3.8/site-packages/distributed/node.py:182: UserWarning: Port 8787 is already in use.
Perhaps you already have a cluster running?
Hosting the HTTP server on port 43945 instead
  warnings.warn(


http://172.16.102.35:43945/status


Task was destroyed but it is pending!
task: <Task pending name='Task-2388543' coro=<HTTP1ServerConnection._server_request_loop() running at /home/soft/anaconda3/envs/chenhj_test_cartopy/lib/python3.8/site-packages/tornado/http1connection.py:825> wait_for=<Future pending cb=[<TaskWakeupMethWrapper object at 0x1543235ab880>()]> cb=[IOLoop.add_future.<locals>.<lambda>() at /home/soft/anaconda3/envs/chenhj_test_cartopy/lib/python3.8/site-packages/tornado/ioloop.py:687]>
Task was destroyed but it is pending!
task: <Task pending name='Task-2338958' coro=<HTTP1ServerConnection._server_request_loop() running at /home/soft/anaconda3/envs/chenhj_test_cartopy/lib/python3.8/site-packages/tornado/http1connection.py:825> wait_for=<Future pending cb=[<TaskWakeupMethWrapper object at 0x1544170d5ee0>()]> cb=[IOLoop.add_future.<locals>.<lambda>() at /home/soft/anaconda3/envs/chenhj_test_cartopy/lib/python3.8/site-packages/tornado/ioloop.py:687]>
Task was destroyed but it is pending!
task: <Task pending 

In [54]:
toshis_ds_detrend_copy.to_netcdf("/home/ys17-23/Extension/personal-data/chenhj/SAM_EAM_data/CMIP6/historical/tmp_var/original/detrend/tos_historical_r144x72_195001-201412.nc")

In [57]:
if __name__ == "__main__":
  with dask.config.set(**{'array.slicing.split_large_chunks': True}):
    client = Client(n_workers=4, threads_per_worker=5,memory_limit='50GB',processes=False)
    print(client.dashboard_link)
    with ProgressBar():
      tosssp585_ds_trend, tosssp585_ds_trend_pvalue, tosssp585_ds_detrend = ca.new_detrend_dim(tosCMIP6_ds_detrend_tmp.sel(time=tosCMIP6_ds_detrend_tmp.time.dt.year>=2015).chunk(dict(time=-1,lon=72,lat=36)))
      tosssp585_p3_ds_trend, tosssp585_p3_ds_trend_pvalue, tosssp585_p3_ds_detrend = ca.new_detrend_dim(tosCMIP6_ds_detrend_tmp.sel(time=tosCMIP6_ds_detrend_tmp.time.dt.year>=2064).chunk(dict(time=-1,lon=72,lat=36)))
      
      tosssp585_ds_detrend.name = "sst"
      tosssp585_p3_ds_detrend.name = "sst"
      tosssp585_ds_detrend_copy = tosssp585_ds_detrend.compute()
      tosssp585_p3_ds_detrend_copy = tosssp585_p3_ds_detrend.compute()
client.close()

http://172.16.102.35:8787/status


Task was destroyed but it is pending!
task: <Task pending name='Task-3370900' coro=<HTTP1ServerConnection._server_request_loop() running at /home/soft/anaconda3/envs/chenhj_test_cartopy/lib/python3.8/site-packages/tornado/http1connection.py:825> wait_for=<Future pending cb=[<TaskWakeupMethWrapper object at 0x15433383fd30>()]> cb=[IOLoop.add_future.<locals>.<lambda>() at /home/soft/anaconda3/envs/chenhj_test_cartopy/lib/python3.8/site-packages/tornado/ioloop.py:687]>
Task was destroyed but it is pending!
task: <Task pending name='Task-3486924' coro=<HTTP1ServerConnection._server_request_loop() running at /home/soft/anaconda3/envs/chenhj_test_cartopy/lib/python3.8/site-packages/tornado/http1connection.py:825> wait_for=<Future pending cb=[<TaskWakeupMethWrapper object at 0x1544eccb08b0>()]> cb=[IOLoop.add_future.<locals>.<lambda>() at /home/soft/anaconda3/envs/chenhj_test_cartopy/lib/python3.8/site-packages/tornado/ioloop.py:687]>
Task was destroyed but it is pending!
task: <Task pending 

In [58]:
tosssp585_ds_detrend_copy.to_netcdf("/home/ys17-23/Extension/personal-data/chenhj/SAM_EAM_data/CMIP6/ssp585/tmp_var/original/detrend/tos_ssp585_r144x72_201501-209912.nc")
tosssp585_p3_ds_detrend_copy.to_netcdf("/home/ys17-23/Extension/personal-data/chenhj/SAM_EAM_data/CMIP6/ssp585/tmp_var/original/detrend/tos_ssp585_p3_r144x72_206401-209912.nc")

In [60]:
del(tosCMIP6_ds_detrend_tmp, toshis_ds_detrend_copy, tosssp585_ds_detrend_copy, tosssp585_p3_ds_detrend_copy)

### 分季节

In [80]:
tosCMIP6_MAM, tosCMIP6_JJA, tosCMIP6_SON, tosCMIP6_DJF = ca.mon_to_season(tosCMIP6_ds.chunk(dict(time=-1)))

/home/soft/anaconda3/envs/chenhj_test_cartopy/lib/python3.8/site-packages/xarray/core/indexing.py:1374: PerformanceWarning: Slicing is producing a large chunk. To accept the large
chunk and silence this warning, set the option
    >>> with dask.config.set(**{'array.slicing.split_large_chunks': False}):
    ...     array[indexer]

To avoid creating the large chunks, set the option
    >>> with dask.config.set(**{'array.slicing.split_large_chunks': True}):
    ...     array[indexer]
  return self.array[key]
/home/ys17-23/chenhj/self_def/cal.py:386: PerformanceWarning: Reshaping is producing a large chunk. To accept the large
chunk and silence this warning, set the option
    >>> with dask.config.set(**{'array.slicing.split_large_chunks': False}):
    ...     array.reshape(shape)

To avoid creating the large chunks, set the option
    >>> with dask.config.set(**{'array.slicing.split_large_chunks': True}):
    ...     array.reshape(shape)Explictly passing ``limit`` to ``reshape`` will also

In [94]:
if __name__ == "__main__":
  with dask.config.set(**{'array.slicing.split_large_chunks': True}):
    client = Client(n_workers=4, threads_per_worker=5,memory_limit='50GB',processes=False)
    print(client.dashboard_link)
    tosCMIP6_MAM, tosCMIP6_JJA, tosCMIP6_SON, tosCMIP6_DJF = dask.compute(tosCMIP6_MAM, tosCMIP6_JJA, tosCMIP6_SON, tosCMIP6_DJF)
client.close()

/home/soft/anaconda3/envs/chenhj_test_cartopy/lib/python3.8/site-packages/distributed/node.py:182: UserWarning: Port 8787 is already in use.
Perhaps you already have a cluster running?
Hosting the HTTP server on port 36001 instead
  warnings.warn(


http://172.16.102.35:36001/status


In [96]:
tosCMIP6_MAM.sel(time=tosCMIP6_MAM.time.dt.year<=2014).to_netcdf("/home/ys17-23/Extension/personal-data/chenhj/SAM_EAM_data/CMIP6/historical/tmp_var/MAM/non_detrend/tos_historical_r144x72_197901-201412.nc")
tosCMIP6_JJA.sel(time=tosCMIP6_JJA.time.dt.year<=2014).to_netcdf("/home/ys17-23/Extension/personal-data/chenhj/SAM_EAM_data/CMIP6/historical/tmp_var/JJA/non_detrend/tos_historical_r144x72_197901-201412.nc")
tosCMIP6_SON.sel(time=tosCMIP6_SON.time.dt.year<=2014).to_netcdf("/home/ys17-23/Extension/personal-data/chenhj/SAM_EAM_data/CMIP6/historical/tmp_var/SON/non_detrend/tos_historical_r144x72_197901-201412.nc")
tosCMIP6_DJF.sel(time=tosCMIP6_DJF.time.dt.year<=2014).to_netcdf("/home/ys17-23/Extension/personal-data/chenhj/SAM_EAM_data/CMIP6/historical/tmp_var/DJF/non_detrend/tos_historical_r144x72_197901-201412.nc")

tosCMIP6_MAM.sel(time=tosCMIP6_MAM.time.dt.year>=2015).to_netcdf("/home/ys17-23/Extension/personal-data/chenhj/SAM_EAM_data/CMIP6/ssp585/tmp_var/MAM/non_detrend/tos_ssp585_r144x72_201501-209912.nc")
tosCMIP6_JJA.sel(time=tosCMIP6_JJA.time.dt.year>=2015).to_netcdf("/home/ys17-23/Extension/personal-data/chenhj/SAM_EAM_data/CMIP6/ssp585/tmp_var/JJA/non_detrend/tos_ssp585_r144x72_201501-209912.nc")
tosCMIP6_SON.sel(time=tosCMIP6_SON.time.dt.year>=2015).to_netcdf("/home/ys17-23/Extension/personal-data/chenhj/SAM_EAM_data/CMIP6/ssp585/tmp_var/SON/non_detrend/tos_ssp585_r144x72_201501-209912.nc")
tosCMIP6_DJF.sel(time=tosCMIP6_DJF.time.dt.year>=2015).to_netcdf("/home/ys17-23/Extension/personal-data/chenhj/SAM_EAM_data/CMIP6/ssp585/tmp_var/DJF/non_detrend/tos_ssp585_r144x72_201501-209912.nc")

tosCMIP6_MAM.sel(time=tosCMIP6_MAM.time.dt.year>=2064).to_netcdf("/home/ys17-23/Extension/personal-data/chenhj/SAM_EAM_data/CMIP6/ssp585/tmp_var/MAM/non_detrend/tos_ssp585_p3_r144x72_206401-209912.nc")
tosCMIP6_JJA.sel(time=tosCMIP6_JJA.time.dt.year>=2064).to_netcdf("/home/ys17-23/Extension/personal-data/chenhj/SAM_EAM_data/CMIP6/ssp585/tmp_var/JJA/non_detrend/tos_ssp585_p3_r144x72_206401-209912.nc")
tosCMIP6_SON.sel(time=tosCMIP6_SON.time.dt.year>=2064).to_netcdf("/home/ys17-23/Extension/personal-data/chenhj/SAM_EAM_data/CMIP6/ssp585/tmp_var/SON/non_detrend/tos_ssp585_p3_r144x72_206401-209912.nc")
tosCMIP6_DJF.sel(time=tosCMIP6_DJF.time.dt.year>=2064).to_netcdf("/home/ys17-23/Extension/personal-data/chenhj/SAM_EAM_data/CMIP6/ssp585/tmp_var/DJF/non_detrend/tos_ssp585_p3_r144x72_206401-209912.nc")

In [84]:
if __name__ == "__main__":
  with dask.config.set(**{'array.slicing.split_large_chunks': True}):
    client = Client(n_workers=4, threads_per_worker=5,memory_limit='50GB',processes=False)
    print(client.dashboard_link)
    with ProgressBar():
      tosCMIP6_MAM_trend, tosCMIP6_MAM_trend_pvalue, tosCMIP6_MAM_detrend = ca.new_detrend_dim(tosCMIP6_MAM.chunk(dict(time=-1,lon=72,lat=36)))

      tosCMIP6_MAM_trend, tosCMIP6_MAM_detrend = dask.compute(tosCMIP6_MAM_trend, tosCMIP6_MAM_detrend)
client.close()

/home/soft/anaconda3/envs/chenhj_test_cartopy/lib/python3.8/site-packages/distributed/node.py:182: UserWarning: Port 8787 is already in use.
Perhaps you already have a cluster running?
Hosting the HTTP server on port 46489 instead
  warnings.warn(


http://172.16.102.35:46489/status


Task was destroyed but it is pending!
task: <Task pending name='Task-18620476' coro=<HTTP1ServerConnection._server_request_loop() running at /home/soft/anaconda3/envs/chenhj_test_cartopy/lib/python3.8/site-packages/tornado/http1connection.py:825> wait_for=<Future pending cb=[<TaskWakeupMethWrapper object at 0x1542c1c04940>()]> cb=[IOLoop.add_future.<locals>.<lambda>() at /home/soft/anaconda3/envs/chenhj_test_cartopy/lib/python3.8/site-packages/tornado/ioloop.py:687]>
Task was destroyed but it is pending!
task: <Task pending name='Task-18621090' coro=<RequestHandler._execute() running at /home/soft/anaconda3/envs/chenhj_test_cartopy/lib/python3.8/site-packages/tornado/web.py:1713> wait_for=<Future pending cb=[<TaskWakeupMethWrapper object at 0x15435c3b87f0>()]> cb=[_HandlerDelegate.execute.<locals>.<lambda>() at /home/soft/anaconda3/envs/chenhj_test_cartopy/lib/python3.8/site-packages/tornado/web.py:2361]>
Task was destroyed but it is pending!
task: <Task pending name='Task-18620477' co

In [89]:
if __name__ == "__main__":
  with dask.config.set(**{'array.slicing.split_large_chunks': True}):
    client = Client(n_workers=4, threads_per_worker=5,memory_limit='50GB',processes=False)
    print(client.dashboard_link)
    with ProgressBar():
      tosCMIP6_MAM_trend, tosCMIP6_MAM_trend_pvalue, tosCMIP6_MAM_detrend = ca.new_detrend_dim(tosCMIP6_MAM.chunk(dict(time=-1,lon=72,lat=36)))

      tosCMIP6_MAM_trend, tosCMIP6_MAM_trend_pvalue, tosCMIP6_MAM_detrend = dask.compute(tosCMIP6_MAM_trend, tosCMIP6_MAM_trend_pvalue,tosCMIP6_MAM_detrend)
      
      tosCMIP6_JJA_trend, tosCMIP6_JJA_trend_pvalue, tosCMIP6_JJA_detrend = ca.new_detrend_dim(tosCMIP6_JJA.chunk(dict(time=-1,lon=72,lat=36)))

      tosCMIP6_JJA_trend, tosCMIP6_JJA_trend_pvalue, tosCMIP6_JJA_detrend = dask.compute(tosCMIP6_JJA_trend, tosCMIP6_JJA_trend_pvalue,tosCMIP6_JJA_detrend)

      tosCMIP6_SON_trend, tosCMIP6_SON_trend_pvalue, tosCMIP6_SON_detrend = ca.new_detrend_dim(tosCMIP6_SON.chunk(dict(time=-1,lon=72,lat=36)))

      tosCMIP6_SON_trend, tosCMIP6_SON_trend_pvalue, tosCMIP6_SON_detrend = dask.compute(tosCMIP6_SON_trend, tosCMIP6_SON_trend_pvalue,tosCMIP6_SON_detrend)

      tosCMIP6_DJF_trend, tosCMIP6_DJF_trend_pvalue, tosCMIP6_DJF_detrend = ca.new_detrend_dim(tosCMIP6_DJF.chunk(dict(time=-1,lon=72,lat=36)))

      tosCMIP6_DJF_trend, tosCMIP6_DJF_trend_pvalue, tosCMIP6_DJF_detrend = dask.compute(tosCMIP6_DJF_trend, tosCMIP6_DJF_trend_pvalue,tosCMIP6_DJF_detrend)
client.close()

/home/soft/anaconda3/envs/chenhj_test_cartopy/lib/python3.8/site-packages/distributed/node.py:182: UserWarning: Port 8787 is already in use.
Perhaps you already have a cluster running?
Hosting the HTTP server on port 46147 instead
  warnings.warn(


http://172.16.102.35:46147/status


2022-12-08 23:29:00,991 - tornado.application - ERROR - Exception in callback <bound method BokehTornado._keep_alive of <bokeh.server.tornado.BokehTornado object at 0x154041c5d400>>
Traceback (most recent call last):
  File "/home/soft/anaconda3/envs/chenhj_test_cartopy/lib/python3.8/site-packages/tornado/ioloop.py", line 921, in _run
    val = self.callback()
  File "/home/soft/anaconda3/envs/chenhj_test_cartopy/lib/python3.8/site-packages/bokeh/server/tornado.py", line 760, in _keep_alive
    c.send_ping()
  File "/home/soft/anaconda3/envs/chenhj_test_cartopy/lib/python3.8/site-packages/bokeh/server/connection.py", line 93, in send_ping
    self._socket.ping(str(self._ping_count).encode("utf-8"))
  File "/home/soft/anaconda3/envs/chenhj_test_cartopy/lib/python3.8/site-packages/tornado/websocket.py", line 444, in ping
    raise WebSocketClosedError()
tornado.websocket.WebSocketClosedError
ERROR:tornado.application:Exception in callback <bound method BokehTornado._keep_alive of <bokeh.

In [92]:
models = tosCMIP6_MAM.coords["models"]
tosCMIP6_MAM_regress = xr.Dataset(
  data_vars=dict(
      slope=(["models", "lat", "lon"], tosCMIP6_MAM_trend[:,:,:,1].data),
      pvalue=(["models", "lat", "lon"], tosCMIP6_MAM_trend_pvalue[:,:,:,1].data),
  ),
  coords=dict(
      models=models.data,
      lat=tosCMIP6_MAM_trend[:,:,:,1].lat.data,
      lon=tosCMIP6_MAM_trend[:,:,:,1].lon.data,
  ),
  attrs=dict(description="long-term trends of MAM tos from 1979-2099"),
)

tosCMIP6_JJA_regress = xr.Dataset(
  data_vars=dict(
      slope=(["models", "lat", "lon"], tosCMIP6_JJA_trend[:,:,:,1].data),
      pvalue=(["models", "lat", "lon"], tosCMIP6_JJA_trend_pvalue[:,:,:,1].data),
  ),
  coords=dict(
      models=models.data,
      lat=tosCMIP6_JJA_trend[:,:,:,1].lat.data,
      lon=tosCMIP6_JJA_trend[:,:,:,1].lon.data,
  ),
  attrs=dict(description="long-term trends of JJA tos from 1979-2099"),
)

tosCMIP6_SON_regress = xr.Dataset(
  data_vars=dict(
      slope=(["models", "lat", "lon"], tosCMIP6_SON_trend[:,:,:,1].data),
      pvalue=(["models", "lat", "lon"], tosCMIP6_SON_trend_pvalue[:,:,:,1].data),
  ),
  coords=dict(
      models=models.data,
      lat=tosCMIP6_SON_trend[:,:,:,1].lat.data,
      lon=tosCMIP6_SON_trend[:,:,:,1].lon.data,
  ),
  attrs=dict(description="long-term trends of SON tos from 1979-2099"),
)

tosCMIP6_DJF_regress = xr.Dataset(
  data_vars=dict(
      slope=(["models", "lat", "lon"], tosCMIP6_DJF_trend[:,:,:,1].data),
      pvalue=(["models", "lat", "lon"], tosCMIP6_DJF_trend_pvalue[:,:,:,1].data),
  ),
  coords=dict(
      models=models.data,
      lat=tosCMIP6_DJF_trend[:,:,:,1].lat.data,
      lon=tosCMIP6_DJF_trend[:,:,:,1].lon.data,
  ),
  attrs=dict(description="long-term trends of DJF tos from 1979-2099"),
)

In [93]:
tosCMIP6_MAM_regress.to_netcdf("/home/ys17-23/Extension/personal-data/chenhj/SAM_EAM_data/CMIP6/ssp585/tmp_var/MAM/non_detrend/tosCMIP6_trend.nc")
tosCMIP6_JJA_regress.to_netcdf("/home/ys17-23/Extension/personal-data/chenhj/SAM_EAM_data/CMIP6/ssp585/tmp_var/JJA/non_detrend/tosCMIP6_trend.nc")
tosCMIP6_SON_regress.to_netcdf("/home/ys17-23/Extension/personal-data/chenhj/SAM_EAM_data/CMIP6/ssp585/tmp_var/SON/non_detrend/tosCMIP6_trend.nc")
tosCMIP6_DJF_regress.to_netcdf("/home/ys17-23/Extension/personal-data/chenhj/SAM_EAM_data/CMIP6/ssp585/tmp_var/DJF/non_detrend/tosCMIP6_trend.nc")

In [98]:
if __name__ == "__main__":
  with dask.config.set(**{'array.slicing.split_large_chunks': True}):
    client = Client(n_workers=4, threads_per_worker=5,memory_limit='50GB',processes=False)
    print(client.dashboard_link)
    with ProgressBar():
      toshis_MAM_trend, toshis_MAM_pvalue, toshis_MAM_detrend = ca.new_detrend_dim(tosCMIP6_MAM_detrend.sel(time=tosCMIP6_MAM_detrend.time.dt.year<=2014).chunk(dict(time=-1)))
      toshis_JJA_trend, toshis_JJA_pvalue, toshis_JJA_detrend = ca.new_detrend_dim(tosCMIP6_JJA_detrend.sel(time=tosCMIP6_JJA_detrend.time.dt.year<=2014).chunk(dict(time=-1)))
      toshis_SON_trend, toshis_SON_pvalue, toshis_SON_detrend = ca.new_detrend_dim(tosCMIP6_SON_detrend.sel(time=tosCMIP6_SON_detrend.time.dt.year<=2014).chunk(dict(time=-1)))
      toshis_DJF_trend, toshis_DJF_pvalue, toshis_DJF_detrend = ca.new_detrend_dim(tosCMIP6_DJF_detrend.sel(time=tosCMIP6_DJF_detrend.time.dt.year<=2014).chunk(dict(time=-1)))
      
      (toshis_MAM_detrend, toshis_JJA_detrend, toshis_SON_detrend, toshis_DJF_detrend) = dask.compute(toshis_MAM_detrend, toshis_JJA_detrend, toshis_SON_detrend, toshis_DJF_detrend)
      
      tosssp585_MAM_trend, tosssp585_MAM_pvalue, tosssp585_MAM_detrend = ca.new_detrend_dim(tosCMIP6_MAM_detrend.sel(time=tosCMIP6_MAM_detrend.time.dt.year>=2015).chunk(dict(time=-1)))
      tosssp585_JJA_trend, tosssp585_JJA_pvalue, tosssp585_JJA_detrend = ca.new_detrend_dim(tosCMIP6_JJA_detrend.sel(time=tosCMIP6_JJA_detrend.time.dt.year>=2015).chunk(dict(time=-1)))
      tosssp585_SON_trend, tosssp585_SON_pvalue, tosssp585_SON_detrend = ca.new_detrend_dim(tosCMIP6_SON_detrend.sel(time=tosCMIP6_SON_detrend.time.dt.year>=2015).chunk(dict(time=-1)))
      tosssp585_DJF_trend, tosssp585_DJF_pvalue, tosssp585_DJF_detrend = ca.new_detrend_dim(tosCMIP6_DJF_detrend.sel(time=tosCMIP6_DJF_detrend.time.dt.year>=2015).chunk(dict(time=-1)))
      
      (tosssp585_MAM_detrend, tosssp585_JJA_detrend, tosssp585_SON_detrend, tosssp585_DJF_detrend) = dask.compute(tosssp585_MAM_detrend, tosssp585_JJA_detrend, tosssp585_SON_detrend, tosssp585_DJF_detrend)
      
      tosssp585_p3_MAM_trend, tosssp585_p3_MAM_pvalue, tosssp585_p3_MAM_detrend = ca.new_detrend_dim(tosCMIP6_MAM_detrend.sel(time=tosCMIP6_MAM_detrend.time.dt.year>=2064).chunk(dict(time=-1)))
      tosssp585_p3_JJA_trend, tosssp585_p3_JJA_pvalue, tosssp585_p3_JJA_detrend = ca.new_detrend_dim(tosCMIP6_JJA_detrend.sel(time=tosCMIP6_JJA_detrend.time.dt.year>=2064).chunk(dict(time=-1)))
      tosssp585_p3_SON_trend, tosssp585_p3_SON_pvalue, tosssp585_p3_SON_detrend = ca.new_detrend_dim(tosCMIP6_SON_detrend.sel(time=tosCMIP6_SON_detrend.time.dt.year>=2064).chunk(dict(time=-1)))
      tosssp585_p3_DJF_trend, tosssp585_p3_DJF_pvalue, tosssp585_p3_DJF_detrend = ca.new_detrend_dim(tosCMIP6_DJF_detrend.sel(time=tosCMIP6_DJF_detrend.time.dt.year>=2064).chunk(dict(time=-1)))
      
      (tosssp585_p3_MAM_detrend, tosssp585_p3_JJA_detrend, tosssp585_p3_SON_detrend, tosssp585_p3_DJF_detrend) = dask.compute(tosssp585_p3_MAM_detrend, tosssp585_p3_JJA_detrend, tosssp585_p3_SON_detrend, tosssp585_p3_DJF_detrend)
client.close()

/home/soft/anaconda3/envs/chenhj_test_cartopy/lib/python3.8/site-packages/distributed/node.py:182: UserWarning: Port 8787 is already in use.
Perhaps you already have a cluster running?
Hosting the HTTP server on port 44247 instead
  warnings.warn(


http://172.16.102.35:44247/status


In [100]:
toshis_MAM_detrend.to_netcdf("/home/ys17-23/Extension/personal-data/chenhj/SAM_EAM_data/CMIP6/historical/tmp_var/MAM/detrend/tos_historical_r144x72_197901-201412.nc")
tosssp585_MAM_detrend.to_netcdf("/home/ys17-23/Extension/personal-data/chenhj/SAM_EAM_data/CMIP6/ssp585/tmp_var/MAM/detrend/tos_ssp585_r144x72_201501-209912.nc")
tosssp585_p3_MAM_detrend.to_netcdf("/home/ys17-23/Extension/personal-data/chenhj/SAM_EAM_data/CMIP6/ssp585/tmp_var/MAM/detrend/tos_ssp585_p3_r144x72_206401-209912.nc")

toshis_JJA_detrend.to_netcdf("/home/ys17-23/Extension/personal-data/chenhj/SAM_EAM_data/CMIP6/historical/tmp_var/JJA/detrend/tos_historical_r144x72_197901-201412.nc")
tosssp585_JJA_detrend.to_netcdf("/home/ys17-23/Extension/personal-data/chenhj/SAM_EAM_data/CMIP6/ssp585/tmp_var/JJA/detrend/tos_ssp585_r144x72_201501-209912.nc")
tosssp585_p3_JJA_detrend.to_netcdf("/home/ys17-23/Extension/personal-data/chenhj/SAM_EAM_data/CMIP6/ssp585/tmp_var/JJA/detrend/tos_ssp585_p3_r144x72_206401-209912.nc")

toshis_SON_detrend.to_netcdf("/home/ys17-23/Extension/personal-data/chenhj/SAM_EAM_data/CMIP6/historical/tmp_var/SON/detrend/tos_historical_r144x72_197901-201412.nc")
tosssp585_SON_detrend.to_netcdf("/home/ys17-23/Extension/personal-data/chenhj/SAM_EAM_data/CMIP6/ssp585/tmp_var/SON/detrend/tos_ssp585_r144x72_201501-209912.nc")
tosssp585_p3_SON_detrend.to_netcdf("/home/ys17-23/Extension/personal-data/chenhj/SAM_EAM_data/CMIP6/ssp585/tmp_var/SON/detrend/tos_ssp585_p3_r144x72_206401-209912.nc")

toshis_DJF_detrend.to_netcdf("/home/ys17-23/Extension/personal-data/chenhj/SAM_EAM_data/CMIP6/historical/tmp_var/DJF/detrend/tos_historical_r144x72_197901-201412.nc")
tosssp585_DJF_detrend.to_netcdf("/home/ys17-23/Extension/personal-data/chenhj/SAM_EAM_data/CMIP6/ssp585/tmp_var/DJF/detrend/tos_ssp585_r144x72_201501-209912.nc")
tosssp585_p3_DJF_detrend.to_netcdf("/home/ys17-23/Extension/personal-data/chenhj/SAM_EAM_data/CMIP6/ssp585/tmp_var/DJF/detrend/tos_ssp585_p3_r144x72_206401-209912.nc")